# **E-Commerce Analytics Pipeline Using FakeStore API**






In [1]:
%%writefile extract.py
import requests
import logging
from typing import List, Dict

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class DataExtractor:
    """
    Extracts product data from FakeStore API with retry and validation handling.
    """

    BASE_URL = "https://fakestoreapi.com/products"

    def __init__(self, retries: int = 3, timeout: int = 10):
        self.retries = retries
        self.timeout = timeout

    def fetch_products(self) -> List[Dict]:
        """
        Fetch all product data from the API.
        Includes error handling and retry logic.
        """
        attempt = 0

        while attempt < self.retries:
            try:
                logging.info(f"Fetching product data... Attempt {attempt + 1}")

                response = requests.get(self.BASE_URL, timeout=self.timeout)

                if response.status_code == 200:
                    logging.info("Data fetched successfully.")
                    return response.json()

                logging.warning(f"Unexpected status code: {response.status_code}")

            except requests.exceptions.RequestException as e:
                logging.error(f"Request failed due to: {e}")

            attempt += 1

        logging.error("Failed to fetch data after multiple retries.")
        return []


Writing extract.py


In [7]:
%%writefile transform.py
import pandas as pd
import logging
from typing import List, Dict

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class DataTransformer:
    """
    Transforms raw product JSON into a clean Pandas DataFrame.
    Adds derived metrics useful for analysis and reporting.
    """

    def json_to_dataframe(self, data: List[Dict]) -> pd.DataFrame:
        """
        Converts JSON into DataFrame with cleaning + new features.
        """

        if not data:
            logging.error("Empty data received for transformation.")
            return pd.DataFrame()

        logging.info("Transforming JSON data into DataFrame...")

        df = pd.DataFrame(data)

        # -------- FLATTEN NESTED COLUMNS --------
        if 'rating' in df.columns:
            df['rating_rate'] = df['rating'].apply(lambda x: x.get('rate') if isinstance(x, dict) else None)
            df['rating_count'] = df['rating'].apply(lambda x: x.get('count') if isinstance(x, dict) else None)
            df.drop(columns=['rating'], inplace=True)

        # -------------------------------
        # Basic cleaning
        # -------------------------------
        df.dropna(subset=['title', 'price'], inplace=True)  # essential fields
        df['category'] = df['category'].str.title()

        # -------------------------------
        # Derived columns
        # -------------------------------
        df['price_with_tax'] = df['price'] * 1.18     # 18% estimated tax
        df['title_length'] = df['title'].str.len()

        # Category price ranking
        df['category_avg_price'] = df.groupby('category')['price'].transform('mean')

        logging.info("Transformation completed successfully.")
        return df


Overwriting transform.py


In [3]:
%%writefile analysis.py
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class DataAnalyzer:
    """
    Performs insights & summary analysis on product data.
    """

    def generate_insights(self, df: pd.DataFrame) -> str:
        """
        Returns a text summary of insights.
        """

        logging.info("Generating insights...")

        insights = []

        insights.append(f"Total products: {len(df)}")
        insights.append(f"Number of categories: {df['category'].nunique()}")

        top_cat = df.groupby('category')['price'].mean().idxmax()
        insights.append(f"Highest avg price category: {top_cat}")

        cheap_prod = df.loc[df['price'].idxmin()]
        insights.append(f"Cheapest product: {cheap_prod['title']} (${cheap_prod['price']})")

        expensive_prod = df.loc[df['price'].idxmax()]
        insights.append(f"Most expensive product: {expensive_prod['title']} (${expensive_prod['price']})")

        return "\n".join(insights)


Writing analysis.py


In [4]:
%%writefile load.py
import pandas as pd
import sqlite3
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class DataLoader:
    """
    Saves cleaned data to CSV and SQLite database.
    """

    def save_to_csv(self, df: pd.DataFrame, filepath: str = "output/products.csv"):
        df.to_csv(filepath, index=False)
        logging.info(f"Data saved to CSV at: {filepath}")

    def save_to_sqlite(self, df: pd.DataFrame, db_path: str = "output/products.db"):
        conn = sqlite3.connect(db_path)
        df.to_sql("products", conn, if_exists="replace", index=False)
        conn.close()
        logging.info(f"Data stored in SQLite DB: {db_path}")



Writing load.py


In [8]:
%%writefile main.py
from extract import DataExtractor
from transform import DataTransformer
from analysis import DataAnalyzer
from load import DataLoader

import os

def ensure_output_folder():
    if not os.path.exists("output"):
        os.makedirs("output")

def main():
    ensure_output_folder()

    # Step 1: Extract
    extractor = DataExtractor()
    raw_data = extractor.fetch_products()

    # Step 2: Transform
    transformer = DataTransformer()
    df = transformer.json_to_dataframe(raw_data)

    if df.empty:
        print("No data to process.")
        return

    # Step 3: Analyze
    analyzer = DataAnalyzer()
    insights = analyzer.generate_insights(df)

    with open("output/summary.txt", "w") as f:
        f.write(insights)

    print("Insights generated:\n", insights)

    # Step 4: Load (CSV + SQLite)
    loader = DataLoader()
    loader.save_to_csv(df)
    loader.save_to_sqlite(df)

    print("\nPipeline executed successfully!")

if __name__ == "__main__":
    main()


Overwriting main.py


In [9]:
!python main.py

2025-11-25 05:48:26,950 - INFO - NumExpr defaulting to 2 threads.
2025-11-25 05:48:27,164 - INFO - Fetching product data... Attempt 1
2025-11-25 05:48:27,440 - INFO - Data fetched successfully.
2025-11-25 05:48:27,441 - INFO - Transforming JSON data into DataFrame...
2025-11-25 05:48:27,449 - INFO - Transformation completed successfully.
2025-11-25 05:48:27,449 - INFO - Generating insights...
Insights generated:
 Total products: 20
Number of categories: 4
Highest avg price category: Electronics
Cheapest product: Opna Women's Short Sleeve Moisture ($7.95)
Most expensive product: Samsung 49-Inch CHG90 144Hz Curved Gaming Monitor (LC49HG90DMNXZA) – Super Ultrawide Screen QLED  ($999.99)
2025-11-25 05:48:27,454 - INFO - Data saved to CSV at: output/products.csv
2025-11-25 05:48:27,492 - INFO - Data stored in SQLite DB: output/products.db

Pipeline executed successfully!
